# Hillslope Mesh

In [1]:
import tinerator as tin
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
%matplotlib widget

# pip install ipympl
# pip install ipywidgets (optional?)
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install jupyter-matplotlib
# jupyter lab build

In [2]:
def f(x, y, xv, zv, v_slope, y0, stream_slope):
    z = zv + v_slope*np.abs(x-xv) + (y-y0)*stream_slope
    return z

def make_mesh(
    stream_slope = 0.05,
    nx=11,
    ny=21,
    x_min=0.0,
    x_max=10.0,
    y_min=0.0,
    y_max=20.0,
    z_min=0.0,
    z_v=7.0,
    z_max=10.0,
):
    x_v = (x_max - x_min) / 2.
    dx = (x_max - x_min) / (nx - 1)
    dy = (y_max - y_min) / (ny - 1)
    
    v_slope = (z_max - z_v) / x_v
    
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)
    
    Z = f(X, Y, x_v, z_v, v_slope, y_min, stream_slope)
    
    return x, y, Z


# The external 'widgets.py' file will
#   compute `make_mesh` with widget functionality
# The parameter `s.xyZ()` returns the x, y, Z data
%run -i widgets.py
mm_widget = MakeMeshWidget()
display(mm_widget)

MakeMeshWidget(children=(VBox(children=(Label(value='Discretization'), IntSlider(value=11, continuous_update=F…

# Use the computed x, y, Z values to construct a hillslope mesh

In [3]:
quad_mesh = tin.meshing.create_hillslope_mesh(
    mm_widget.Z, x_coords=mm_widget.x, y_coords=mm_widget.y
)
quad_mesh.view()

[00:13:25] Hillslope and quad meshes are an experimental feature.              ]8;id=1630044805.1403148-548982;file:///Users/livingston/dev/lanl/tinerator/tinerator/logging.py\logging.py]8;;\:126

           Expect things to break.                                             ]8;id=1630044805.144335-100133;file:///Users/livingston/dev/lanl/tinerator/tinerator/logging.py\logging.py]8;;\:126

# Extrude from a quad surface mesh to a hexahedron volume mesh

In [4]:
z_min = 0.0
n_z = 11

layers = [("snapped", z_min, n_z, 1),]
hex_mesh = tin.meshing.extrude_mesh(quad_mesh, layers)
hex_mesh.view()

# Create face sets and write to ExodusII file

In [5]:
surface_mesh = hex_mesh.surface_mesh()

set_top = surface_mesh.top_faces
sets_normals = surface_mesh.from_cell_normals()

sets = sets_normals + [set_top]

hex_mesh.view(sets=sets)
hex_mesh.save("Open-Book-3D.exo", sets=sets)

north: min = 191; max = 2200
south: min = 1; max = 2010
east: min = 10; max = 2200
west: min = 1; max = 2191
down: min = 2001; max = 2200
TopFaces: min = 1; max = 200

You are using exodus.py v 1.20.2 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: Open-Book-3D.exo
Closing exodus file: Open-Book-3D.exo
EXODUS write was successful.
